In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [2]:
class Dataset_weather(Dataset):
    def __init__(self,path,seqs=5,preds=1,scale=True) -> None:
        super().__init__()
        self.data_path = path
        self.scale = scale
        self.seqs = seqs # 每个样本中的图片序列长度
        self.preds = preds # 每个样本预测图片的数量
        self.__read_data__() # 这里读取后的数据形状都是(self.channels,self.new_L)即横置的

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(self.data_path,index_col=0) # shape:(52696, 21)
        data_array = df_raw.to_numpy()
        # 截断数据使其为C的倍数
        L, C = data_array.shape
        self.channels = C
        new_L = (L // C ) * C 
        self.new_L = new_L
        trun_data_array = data_array[:new_L,:]
        assert len(trun_data_array) % C == 0
        # 数据标准化
        if self.scale:
            self.scaler.fit(trun_data_array)
            data = self.scaler.transform(trun_data_array)
        else:
            data = trun_data_array
        
        dataT = data.T
        # print(dataT.shape) (C,new_L)        
        self.data_x = dataT
        # self.data_y = dataT
       
    def __getitem__(self, index):
        C,L = self.channels,self.new_L
        
        seq_x = self.data_x[:,index*C:(index+self.seqs)*C]
        y_true = self.data_x[:,(index+self.seqs)*C:(index+self.seqs+1)*C]

        return seq_x,y_true

    def __len__(self):
        return int((self.new_L / self.channels) )- self.seqs
    
    def inverse_transform(self, data):
        data = data.T
        inverse_trans_data = self.scaler.inverse_transform(data)
        return inverse_trans_data.T


In [3]:
dataset = Dataset_weather(path=r'D:\时序数据集\all_datasets\all_datasets\weather\weather.csv')

In [4]:
x0,y0 = dataset[0]
print(x0.shape)
print(y0.shape)
x1,y1 = dataset[1]
print(x1.shape)
print(y1.shape)

(21, 105)
(21, 21)
(21, 105)
(21, 21)


In [11]:
data_test_loader = DataLoader(dataset,batch_size=4,shuffle=False,drop_last=False)

In [ ]:
for batch_x,batch_y in data_test_loader:
    print(batch_y.shape)
    batch_y =batch_y.detach().cpu().numpy()
    # 转置数据，使得形状变为 (variable_channels, time_steps, batch_size)
    data_transposed = batch_y.transpose(1, 2, 0)  # 变为 (21, 21, 4)
    # 然后调整形状，将其变为 (variable_channels, time_steps * batch_size)
    result = data_transposed.reshape(21, 21 * 4)

    trues_inv = dataset.inverse_transform(result)
    print(trues_inv.shape) # (21,84)
    break


torch.Size([4, 21, 21])
(21, 84)


In [5]:
data = dataset.inverse_transform(x0)
data.shape

(21, 105)